Cliente

In [92]:
try:
    # Tentativa de listar modelos para verificar a chave
    models = client.models.list()
    print("✅ Chave OpenAI carregada e funcionando corretamente!")
    # Você pode descomentar a linha abaixo para ver a lista de modelos se desejar
    # for model in models.data:
    #     print(model.id)
except Exception as e:
    print(f"❌ Erro ao verificar a chave OpenAI: {e}")
    print("Por favor, verifique se sua chave OPENAI_API_KEY está salva corretamente no Secrets Manager do Colab.")

✅ Chave OpenAI carregada e funcionando corretamente!


In [93]:
# @title
import os
!pip install openai
from openai import OpenAI
from google.colab import userdata

# Configurar chave (já criada no painel da OpenAI)
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

Etapa 1 – Preparação

In [97]:
import random, time
from IPython.display import Audio, display

acertos = 0

def speak_and_print(text, filename_prefix="message", visual_prefix=""):
    """Generates speech from text, plays it, and prints the text, optionally with a visual prefix."""
    try:
        tts = client.audio.speech.create(
            model="gpt-4o-mini-tts",
            voice="alloy",
            input=text
        )
        audio_filename = f"{filename_prefix}_{int(time.time() * 1000)}.mp3"
        with open(audio_filename, "wb") as f:
            f.write(tts.read())
        display(Audio(audio_filename, autoplay=True))
        time.sleep(len(text) / 20) # Give some time for audio to play
    except Exception as e:
        print(f"[ERRO DE ÁUDIO] Não foi possível reproduzir: {text} - Erro: {e}")

    print(f"{visual_prefix} {text}")

def cronometro(segundos):
    print(f"Você tem {segundos} segundos para responder...")
    for i in range(segundos, 0, -1):
        print(f"Tempo restante: {i} segundos", end='\r')
        time.sleep(1)
    print("Tempo esgotado!")

Etapa 2 – Pergunta em Áudio

In [98]:
def pergunta_audio(pergunta):
    tts_pergunta = client.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="alloy",
        input=pergunta
    )
    with open("pergunta.mp3", "wb") as f:
        f.write(tts_pergunta.read())
    display(Audio("pergunta.mp3", autoplay=True))
    print(f"❓ Pergunta: {pergunta}") # Ocultado a pedido do usuário

Etapa 3 - Resposta do aluno

In [99]:
def resposta_aluno():
    audio_data = record_audio_in_colab()

    # Save the recorded audio to a file
    audio_filename = "resposta.webm" # Using webm format from MediaRecorder
    with open(audio_filename, "wb") as f:
        f.write(audio_data)

    try:
        with open(audio_filename, "rb") as audio_file:
            transcricao = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
        resposta = transcricao.text
    except Exception as e:
        print(f"❌ Ocorreu um erro ao processar o áudio: {e}")
        resposta = ""

    print("📝 Resposta do aluno:", resposta)
    return resposta

Etapa 4 – Avaliação da Resposta

In [100]:
def avaliar_resposta(pergunta, resposta_correta, resposta_aluno):
    avaliacao = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Você é um professor didático e atencioso. Ao avaliar a resposta do aluno, dirija-se a ele diretamente, usando a primeira ou segunda pessoa. Se a resposta do aluno estiver *essencialmente correta* (mesmo que resumida ou sem detalhes completos), classifique-a como 'correta' e *em seguida* ofereça sugestões sobre como ela poderia ser mais completa e satisfatória. Se a resposta estiver *parcialmente correta* ou *incorreta* por falhas factuais, explique detalhadamente o porquê, o que faltou ou está errado, e como o aluno poderia responder de forma mais completa. Se a resposta do aluno não tiver nenhuma relação com a pergunta ou for completamente irrelevante, classifique-a como incorreta, sem tentar encontrar qualquer conexão."},
            {"role": "user", "content": f"Pergunta: {pergunta}\nResposta correta: {resposta_correta}\nResposta do aluno: {resposta_aluno}"}
        ]
    )
    return avaliacao.choices[0].message.content

Etapa 5 – Feedback em Áudio

In [101]:
def feedback_audio(pergunta, feedback):
    global acertos

    # The question is spoken by pergunta_audio, so no need to speak it again here.
    # print(f"❓ Pergunta sorteada: {pergunta}")

    tts_feedback = client.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="alloy",
        input=feedback
    )
    with open("feedback_aluno.mp3", "wb") as f:
        f.write(tts_feedback.read())
    display(Audio("feedback_aluno.mp3", autoplay=True))
    # Only speak feedback, not the meta-message about it being reproduced
    # speak_and_print("Feedback reproduzido.", visual_prefix="🔊", filename_prefix="feedback_reproduzido")

    # Refined logic to count only truly correct answers
    if "sua resposta está correta" in feedback.lower() and "parcialmente correta" not in feedback.lower():
        acertos += 1

    time.sleep(2)
    # Speak the prompt for continuing
    # speak_and_print("Deseja continuar com outra pergunta? (sim/não)", filename_prefix="prompt_continuar")
    resposta_usuario = input("Deseja continuar com outra pergunta? (sim/não)").strip().lower()
    return resposta_usuario

Etapa 6 – Fluxo Principal

In [102]:
from IPython.display import Javascript, display
from google.colab.output import eval_js
from base64 import b64decode

def record_audio_in_colab():
    js_code = '''
        const sleep = ms => new Promise(resolve => setTimeout(resolve, ms));
        const record = async () => {
            const options = { mimeType: 'audio/webm; codecs=opus' };
            const div = document.createElement('div');

            const instruction = document.createElement('p');
            instruction.textContent = '🎤 Clique no Botão para Iniciar a gravação da sua resposta.';
            instruction.style.fontWeight = 'bold';
            div.appendChild(instruction);

            const capture = document.createElement('button');
            const stop = document.createElement('button');
            let stream;
            let mediaRecorder;
            let audioData = [];
            let onstopPromiseResolve;

            capture.textContent = '▶️ Iniciar Gravação';
            capture.style.background = '#4CAF50';
            capture.style.color = 'white';
            capture.style.border = 'none';
            capture.style.padding = '10px 20px';
            capture.style.margin = '5px';
            capture.style.borderRadius = '5px';
            capture.style.cursor = 'pointer';

            stop.textContent = '■ Parar Gravação';
            stop.style.background = '#f44336';
            stop.style.color = 'white';
            stop.style.border = 'none';
            stop.style.padding = '10px 20px';
            stop.style.margin = '5px';
            stop.style.borderRadius = '5px';
            stop.style.cursor = 'pointer';
            stop.style.display = 'none'; // Initially hidden

            div.appendChild(capture);
            div.appendChild(stop);
            document.body.appendChild(div);

            const recordingStopped = new Promise(resolve => { onstopPromiseResolve = resolve; });

            capture.onclick = async () => {
                stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                mediaRecorder = new MediaRecorder(stream, options);
                mediaRecorder.ondataavailable = e => audioData.push(e.data);
                mediaRecorder.onstop = () => {
                    stream.getTracks().forEach(track => track.stop());
                    capture.style.display = 'inline-block';
                    stop.style.display = 'none';
                    div.remove(); // Remove buttons and instruction after recording
                    onstopPromiseResolve();
                };
                mediaRecorder.start();
                capture.style.display = 'none';
                stop.style.display = 'inline-block';
            };

            stop.onclick = () => {
                if (mediaRecorder && mediaRecorder.state !== 'inactive') {
                    mediaRecorder.stop();
                }
            };

            await recordingStopped;

            const blob = new Blob(audioData, options);

            // Convert blob to base64
            const reader = new FileReader();
            reader.readAsDataURL(blob);
            await new Promise(resolve => reader.onloadend = resolve);
            return reader.result.split(',')[1];
        };
        record(); // Call the record function immediately
    '''
    data_b64 = eval_js(js_code, timeout_sec=600) # Increased timeout for recording
    return b64decode(data_b64)

In [103]:
def iniciar_fluxo():
    import json # Import json locally within the function
    import time # Import time for timestamp
    import random # Import random for shuffling, already present but good to confirm
    import unicodedata # Added for accent removal
    import re # Added for regex in accent removal
    from IPython.display import Audio # Added Audio import

    # Helper function to remove accents from a string
    def remove_accents(input_str):
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Initialize variables for the first quiz
    assunto = ""
    num_questoes = 0
    nivel_questoes = ""
    last_qa_pairs = [] # To store the last generated set for reuse

    primeira_rodada = True # Flag for initial full parameter input
    regenerate_questions_next_round = True # Flag to control GPT generation

    while True: # Outer loop for multiple quiz sessions
        # --- Collect/Reconfirm quiz parameters ---
        if primeira_rodada:
            assunto = input("Digite o assunto que deseja estudar: ").strip().lower()

            while True:
                try:
                    num_questoes = int(input("Quantas questões você deseja responder? "))
                    if isinstance(num_questoes, int) and num_questoes > 0:
                        break
                    else:
                        print("❌ Por favor, digite um número válido e maior que zero.")
                except ValueError:
                    print("❌ Por favor, digite um número válido.")

            while True:
                raw_nivel_questoes = input("Qual o nível das questões? (fácil, médio ou difícil): ").strip().lower()
                # Normalize the input to remove accents for validation
                normalized_input = remove_accents(raw_nivel_questoes)

                # Compare against normalized valid options
                if normalized_input in [remove_accents(n) for n in ["fácil", "médio", "difícil"]]:
                    # Keep the original input for the GPT prompt as it might prefer accented versions
                    nivel_questoes = raw_nivel_questoes
                    break
                else:
                    print("❌ Por favor, selecione um nível válido: fácil, médio ou difícil.")

            primeira_rodada = False # Next time, it won't be the first round
            regenerate_questions_next_round = True # Always generate new for a completely new config

        # The 'else' block for parameter reconfirmation is removed here
        # because 'final_choice' will now directly control the next state.
        # If 'rodada' is chosen, parameters are kept and new questions are forced.
        # If 'novo' is chosen, parameter input is forced from scratch.

        # --- Question Generation (Conditional) ---
        qa_pairs = [] # Reset for current round
        if regenerate_questions_next_round:
            # Add a timestamp to the prompt to encourage new questions even with same parameters
            unique_prompt_suffix = f" (Ref: {int(time.time())})"; # Renamed to 'Ref' to make it sound more like a unique reference

            geracao = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                   {"role": "system", "content": "Responda apenas com JSON válido, usando aspas duplas em todas as chaves e valores, sem explicações."},
                   {"role": "user", "content": f'Gerar {num_questoes} perguntas e respostas curtas sobre {assunto}, com nível {nivel_questoes}. IMPORTANTE: As perguntas devem ser *sempre* fundamentalmente novas e diferentes umas das outras, e diferentes de qualquer pergunta que você possa ter gerado anteriormente sobre este tópico, mesmo que o assunto e o nível sejam os mesmos. Use esta referência única para garantir a novidade: {unique_prompt_suffix}. Retorne como uma lista JSON no formato: [{{"pergunta":"...","resposta":"..."}}]'}
                ]
            )
            conteudo = geracao.choices[0].message.content
            # print("\n🔍 Conteúdo retornado pelo GPT:\n", conteudo) # Ocultado a pedido do usuário

            try:
                qa_pairs = json.loads(conteudo)
                print("✅ Banco de questões gerado com sucesso!")
                last_qa_pairs = qa_pairs # Store the newly generated questions
            except json.JSONDecodeError:
                print("❌ Erro ao converter para JSON. O retorno não está no formato esperado.")
                qa_pairs = []
            regenerate_questions_next_round = False # Questions have been generated for this round
        else: # Reuse existing questions, if regenerate_questions_next_round is False
            if last_qa_pairs:
                qa_pairs = list(last_qa_pairs) # Get a fresh copy to ensure shuffling works on the full set
                print("✅ Reutilizando banco de questões anterior!")
            else:
                print("❌ Não há questões anteriores para reutilizar. Gerando novas questões.")
                # This case should ideally not be hit if flags are set correctly, but as a fallback
                regenerate_questions_next_round = True # Force generation if reuse fails
                continue # Restart outer loop to generate questions

        if not qa_pairs:
            speak_and_print("Não foi possível gerar questões com os parâmetros fornecidos. Por favor, tente novamente.", filename_prefix="erro_geracao") # Keeping this speak_and_print for critical error
            primeira_rodada = True # Force re-input of parameters
            regenerate_questions_next_round = True # Also force new generation
            continue # Restart the outer loop

        # --- Execute quiz round ---
        global acertos
        acertos = 0 # Reset acertos for each new quiz session
        total = num_questoes # Use the locally defined num_questoes for the current quiz
        percentual_aprovacao = 70 # Default approval percentage

        # Shuffle the qa_pairs to ensure questions are asked without repetition
        perguntas_restantes = list(qa_pairs)
        random.shuffle(perguntas_restantes)

        contador = 0
        while contador < total and perguntas_restantes: # Loop until all 'total' questions are asked or user stops
            sorteada = perguntas_restantes.pop(0) # Get and remove the first question from the shuffled list
            pergunta, resposta_correta = sorteada["pergunta"], sorteada["resposta"]

            # Pergunta em áudio
            pergunta_audio(pergunta)

            # Cronômetro (ex: 10 segundos)
            cronometro(10)

            # Resposta do aluno
            resposta = resposta_aluno()

            # Avaliação
            feedback = avaliar_resposta(pergunta, resposta_correta, resposta)
            print("📊 Avaliação:\n", feedback)

            # Feedback em áudio
            resposta_continuar = feedback_audio(pergunta, feedback)

            if resposta_continuar == "não" or resposta_continuar == "nao":
                break # Break out of the inner question loop

            contador += 1

        # --- Quiz round ended. Present score and ask about next session ---
        score_message = f"Você acertou {acertos} de {total} questões."
        print(f"📊 {score_message}") # Changed speak_and_print to print

        percentual_acertos = (acertos / total) * 100
        percentual_erros = 100 - percentual_acertos

        final_message = ""
        if acertos == total:
            final_message = "Parabéns! Você acertou todas as questões!"
            print(f"🎉 {final_message}") # Changed speak_and_print to print
        elif percentual_acertos >= percentual_aprovacao:
            final_message = f"Muito bom! Você atingiu {percentual_acertos:.1f}% de acertos e foi aprovado!"
            print(f"👏 {final_message}") # Changed speak_and_print to print
        else:
            final_message = f"Você acertou {percentual_acertos:.1f}% e errou {percentual_erros:.1f}%. Continue estudando para melhorar!"
            print(f"💡 {final_message}") # Changed speak_and_print to print

        # Ask if user wants to continue or stop
        final_choice = input("O quiz terminou. Deseja iniciar uma nova rodada (com as mesmas configurações), configurar um novo quiz completamente, ou parar? (rodada/novo/parar)").strip().lower()

        if final_choice == 'rodada':
            primeira_rodada = False # Keep parameters, but next iteration won't be 'first round'
            regenerate_questions_next_round = True # Force new questions with same parameters
            continue # Continue outer loop
        elif final_choice == 'novo':
            primeira_rodada = True # Force full parameter re-input in next iteration
            regenerate_questions_next_round = True # New config implies new questions
            continue # Continue outer loop
        elif final_choice == 'parar':
            break # Break out of the outer `while True` loop to go to doubt section
        else:
            print("Opção inválida. Por favor, digite 'rodada', 'novo' ou 'parar'.") # Changed speak_and_print to print
            # If invalid, just break to doubt section for now to avoid infinite loop with bad input.
            break

    # --- Doubt section (after all quiz sessions are explicitly ended) ---
    while True:
        print("Você gostaria de tirar alguma dúvida sobre o assunto? (sim/não)") # This prints the prompt visually
        time.sleep(2) # Adicionado para garantir que o áudio do prompt toque antes do input
        tirar_duvida = input().strip().lower() # Removed the redundant prompt from input()
        if tirar_duvida == "sim":
            print("🎤 Clique nos botões para gravar sua dúvida.") # Changed speak_and_print to print
            audio_data_duvida = record_audio_in_colab()

            # Save the recorded audio to a file
            audio_filename_duvida = "duvida.webm"
            with open(audio_filename_duvida, "wb") as f:
                f.write(audio_data_duvida)

            try:
                with open(audio_filename_duvida, "rb") as audio_file:
                    transcricao_duvida = client.audio.transcriptions.create(
                        model="whisper-1",
                        file=audio_file
                    )
                duvida_aluno = transcricao_duvida.text
                print(f"📝 Sua dúvida (transcrita): {duvida_aluno}")
            except Exception as e:
                print(f"❌ Ocorreu um erro ao processar o áudio da dúvida: {e}")
                duvida_aluno = ""

            if duvida_aluno:
                feedback_gpt_text = ""
                try:
                    # Gerar feedback para a dúvida usando o GPT
                    response_gpt_feedback = client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=[
                            {"role": "system", "content": "Você é um professor didático e atencioso. Responda à dúvida do aluno de forma clara e útil."},
                            {"role": "user", "content": f"Dúvida do aluno sobre {assunto}: {duvida_aluno}"}
                        ]
                    )
                    feedback_gpt_text = response_gpt_feedback.choices[0].message.content
                except Exception as e:
                    print(f"❌ Ocorreu um erro ao gerar o feedback da dúvida com o GPT: {e}")
                    feedback_gpt_text = "Desculpe, não consegui gerar um feedback para sua dúvida no momento. Por favor, tente novamente."

                # Generate and play audio for the feedback on the doubt once
                tts_feedback_duvida = client.audio.speech.create(
                    model="gpt-4o-mini-tts",
                    voice="alloy",
                    input=feedback_gpt_text
                )
                with open("feedback_duvida_spoken.mp3", "wb") as f:
                    f.write(tts_feedback_duvida.read())
                display(Audio("feedback_duvida_spoken.mp3", autoplay=True))
                print(f"📚 Feedback sobre sua dúvida:\n{feedback_gpt_text}") # Print the text feedback AFTER audio
                time.sleep(3) # Give time for audio to play before next interaction and to read text

            else:
                print("Não foi possível processar sua dúvida de áudio.")

        elif tirar_duvida == "não" or tirar_duvida == "nao":
            speak_and_print("Obrigado por participar! Até a próxima. Continue se dedicando aos estudos!", filename_prefix="despedida_final")
            break
        else:
            print("Opção inválida. Por favor, digite 'sim' ou 'não'.") # Changed speak_and_print to print

Etapa 7 - Iniciar Fluxo

In [104]:
iniciar_fluxo()

Digite o assunto que deseja estudar: agua
Quantas questões você deseja responder? 2
Qual o nível das questões? (fácil, médio ou difícil): fácil
✅ Banco de questões gerado com sucesso!


❓ Pergunta: Qual é a fórmula química da água?
Você tem 10 segundos para responder...
Tempo esgotado!
📝 Resposta do aluno: Thanks for watching!
📊 Avaliação:
 Sua resposta está incorreta. Embora você tenha dado um agradecimento, não respondeu à pergunta sobre a fórmula química da água. A resposta correta é H2O. Para melhorar sua resposta, você poderia simplesmente ter afirmado que a fórmula química da água é H2O e possivelmente acrescentar uma breve explicação sobre o que isso significa em termos de composição química. Se precisar de mais ajuda com este tópico, estou aqui para ajudar!


Deseja continuar com outra pergunta? (sim/não)sim


❓ Pergunta: A água é essencial para quais funções do corpo humano?
Você tem 10 segundos para responder...
Tempo esgotado!
📝 Resposta do aluno: Thanks for watching!
📊 Avaliação:
 Sua resposta está incorreta. O que você apresentou não é uma resposta relacionada à pergunta sobre as funções da água no corpo humano. Para responder corretamente, você poderia mencionar como a água é vital para a hidratação, ajudando a regular a temperatura do corpo por meio da sudorese e da respiração, e também como atua no transporte de nutrientes e na remoção de resíduos através da urina. Tente focar nessas funções e desenvolvê-las um pouco mais na próxima vez!


Deseja continuar com outra pergunta? (sim/não)não
📊 Você acertou 0 de 2 questões.
💡 Você acertou 0.0% e errou 100.0%. Continue estudando para melhorar!
O quiz terminou. Deseja iniciar uma nova rodada (com as mesmas configurações), configurar um novo quiz completamente, ou parar? (rodada/novo/parar)parar
Você gostaria de tirar alguma dúvida sobre o assunto? (sim/não)
não


 Obrigado por participar! Até a próxima. Continue se dedicando aos estudos!
